In [69]:
import pandas as pd
import numpy as np

from pathlib import Path

from sklearn.model_selection import GridSearchCV, train_test_split, KFold, cross_val_score

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, make_scorer

def rmse_score(y_true, y_pred):
    return float(np.sqrt(mean_squared_error(y_true, y_pred)))

rmse_scorer = make_scorer(rmse_score, greater_is_better=False)

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor


In [70]:
#data
path = "training/players.csv"
fantasy_data = pd.read_csv(path)

fantasy_data_2024 = fantasy_data[fantasy_data["Next_Season"] == 2024]
fantasy_data = fantasy_data[fantasy_data["Next_Season"] != 2024]

target_col = "Fantasy_PPR_NextYear"
dropped_cols = {target_col, "Unnamed: 0", "Name", "Team", "Position", "Team_NextYear"}

QB_data = fantasy_data[fantasy_data["Position"] == "QB"] .drop(columns = dropped_cols)
RB_data = fantasy_data[fantasy_data["Position"] == "RB"].drop(columns = dropped_cols)
WR_data = fantasy_data[fantasy_data["Position"] == "WR"].drop(columns = dropped_cols)
TE_data = fantasy_data[fantasy_data["Position"] == "TE"].drop(columns = dropped_cols)

QB_y = fantasy_data.loc[QB_data.index, target_col]
RB_y = fantasy_data.loc[RB_data.index, target_col]
WR_y = fantasy_data.loc[WR_data.index, target_col]
TE_y = fantasy_data.loc[TE_data.index, target_col]

QB_data_2024 = fantasy_data_2024[fantasy_data_2024["Position"] == "QB"].drop(columns = dropped_cols)
RB_data_2024 = fantasy_data_2024[fantasy_data_2024["Position"] == "RB"].drop(columns = dropped_cols)
WR_data_2024 = fantasy_data_2024[fantasy_data_2024["Position"] == "WR"].drop(columns = dropped_cols)
TE_data_2024 = fantasy_data_2024[fantasy_data_2024["Position"] == "TE"].drop(columns = dropped_cols)

In [71]:
#train/test/validation split

X_train_QB, X_test_QB, y_train_QB, y_test_QB = train_test_split(QB_data, QB_y, test_size = 0.20, random_state = 4)

X_train_RB, X_test_RB, y_train_RB, y_test_RB = train_test_split(RB_data, RB_y, test_size = 0.20, random_state = 4)

X_train_WR, X_test_WR, y_train_WR, y_test_WR = train_test_split(WR_data, WR_y, test_size = 0.20, random_state = 4)

X_train_TE, X_test_TE, y_train_TE, y_test_TE = train_test_split(TE_data, TE_y, test_size = 0.20, random_state = 4)

In [72]:
#models

model_pipeline = {
    "Random Forest": (
        RandomForestRegressor(n_jobs = 1, random_state = 4),
        {
            "model__n_estimators": [200, 350, 500],
            "model__max_depth": [5, 10, 20],
            "model__min_samples_split": [2, 5, 10],
        }, 
    ), 

    "Lasso": (
       Lasso(max_iter = 50000, random_state = 4),
        {
            "model__alpha": [.01, .1, 1, 10]
        }, 
    ), 

    "Neural Network": (
        MLPRegressor(max_iter = 4000, early_stopping = True, random_state = 4),
        {
            "model__hidden_layer_sizes": [(64, ), (128, ), (64, 32), (128, 64)],
            "model__alpha": [.0001, .001, .01],
            "model__learning_rate_init": [.001, .01, .1],
        }, 
    ), 
    "Support Vector Regression": (
        SVR(),
        {
            "model__C": [.1, 1, 10],
            "model__gamma": ["scale", "auto"],
            "model__kernel": ["rbf", "linear"],
        }, 
    ), 
    "XGBoost": (
        XGBRegressor(n_jobs = -1, tree_method = "hist", n_estimators = 300, objective = "reg:squarederror", random_state = 4),
        {
            "model__n_estimators": [200, 350, 500],
            "model__max_depth": [3, 5, 7],
            "model__learning_rate": [.001, .01, .1],
            "model__subsample": [.5, .8, 1],
            "model__colsample_bytree": [.5, .8, 1]
        }, 
    ), 
}


In [73]:
#backward feature elimination

def backward_feature_elimination(model, X_train, y_train, X_test, y_test, min_features = 5):

    included_features = list(X_train.columns)
    
    model.fit(X_train[included_features], y_train)
    y_pred = model.predict(X_test[included_features])
    best_rmse = rmse_score(y_test, y_pred)
    best_features = included_features.copy()

    predictor = model.named_steps["model"]

    if hasattr(predictor, "feature_importances_"):
        def important():
            return np.asarray(model.named_steps["model"].feature_importances_)   
    elif hasattr(predictor, "coef_"):
        def important():
            return np.abs(np.ravel(model.named_steps["model"].coef_))  
    else:
        return best_features, best_rmse

    while len(included_features) > min_features:
        importances = important()
        dropped_index = int(np.argmin(importances))
        dropped_feature = included_features[dropped_index]
        
        current = included_features.copy()
        current.remove(dropped_feature)
        model.fit(X_train[current], y_train)
        y_pred = model.predict(X_test[current])
        current_rmse = rmse_score(y_test, y_pred)

        if current_rmse <= best_rmse:
            included_features = current
            best_rmse = current_rmse
            best_features = current.copy()
            
            predictor = model.named_steps["model"]
        else:
            break
        
    return best_features, best_rmse

In [74]:
#QB Tuning

QB_results = []

for name, (base_model, param_grid) in model_pipeline.items():
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy = "median")),
        ("scaler", StandardScaler()),
        ("model", base_model)
        ])
    
    grid = GridSearchCV(pipe, param_grid = param_grid, scoring = rmse_scorer, cv = 3, n_jobs = -1)
    grid.fit(X_train_QB, y_train_QB)

    best_model = grid.best_estimator_
    best_parameters = grid.best_params_

    best_features, test_rmse = backward_feature_elimination(best_model, X_train_QB, y_train_QB, X_test_QB, y_test_QB)

    QB_results.append({
        "Model": name, 
        "Best Parameters": best_parameters,
        "Selected Features": len(best_features),
        "Test_rmse": test_rmse})
    
QB_results_df = pd.DataFrame(QB_results).sort_values("Test_rmse")
QB_results_df


/Users/bunch/opt/anaconda3/lib/python3.9/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.691e+02, tolerance: 2.170e+02
  model = cd_fast.enet_coordinate_descent(
/Users/bunch/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


,Model,Best Parameters,Selected Features,Test_rmse
4,XGBoost,"{'model__colsample_bytree': 0.8, 'model__learning_rate': 0.01, 'model__max_depth': 3, 'model__n_estimators': 200, 'model__subsample': 0.5}",22,77.635020
1,Lasso,{'model__alpha': 10},5,79.558447
3,Support Vector Regression,"{'model__C': 10, 'model__gamma': 'scale', 'model__kernel': 'linear'}",22,79.715159
0,Random Forest,"{'model__max_depth': 5, 'model__min_samples_split': 10, 'model__n_estimators': 500}",22,79.959690
2,Neural Network,"{'model__alpha': 0.01, 'model__hidden_layer_sizes': (128, 64), 'model__learning_rate_init': 0.1}",22,87.357263


In [75]:
#QB predictions

best_model = grid.best_estimator_

best_features, test_rmse = backward_feature_elimination(best_model, X_train_QB, y_train_QB, X_test_QB, y_test_QB)

best_model.fit(X_train_QB[best_features], y_train_QB)

y_pred_QB = best_model.predict(X_test_QB[best_features])


QB_test_results = pd.DataFrame({
    "Actual_Points": y_test_QB,
    "Predicted_Points": y_pred_QB
}, index = X_test_QB.index)

QB_best_model = best_model
QB_best_features = best_features

QB_test_results = fantasy_data.loc[QB_test_results.index, ["Name", "Team", "Position"]].join(QB_test_results)
QB_test_results

,Name,Team,Position,Actual_Points,Predicted_Points
1624,Kirk Cousins,MIN,QB,149.7,211.912415
257,Brandon Allen,DEN,QB,49.7,44.504829
1144,Jameis Winston,NOR,QB,43.9,100.258278
946,Joshua Dobbs,PIT,QB,20.8,39.899506
807,Chad Henne,KAN,QB,3.3,57.304558
1046,Justin Herbert,LAC,QB,281.3,231.625687
1651,Derek Carr,LVR,QB,241.1,185.716171
47,Patrick Mahomes,KAN,QB,374.4,255.805573
82,Baker Mayfield,CLE,QB,248.6,192.430176
527,Ryan Tannehill,TEN,QB,268.4,256.275970


In [76]:
#RB Tuning

RB_results = []

for name, (base_model, param_grid) in model_pipeline.items():
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy = "median")),
        ("scaler", StandardScaler()),
        ("model", base_model)
        ])
    
    grid = GridSearchCV(pipe, param_grid = param_grid, scoring = rmse_scorer, cv = 3, n_jobs = -1)
    grid.fit(X_train_RB, y_train_RB)

    best_model = grid.best_estimator_
    best_parameters = grid.best_params_

    best_features, test_rmse = backward_feature_elimination(best_model, X_train_RB, y_train_RB, X_test_RB, y_test_RB)

    RB_results.append({
        "Model": name, 
        "Best Parameters": best_parameters,
        "Selected Features": len(best_features),
        "Test_rmse": test_rmse})
    
    
RB_results_df = pd.DataFrame(RB_results).sort_values("Test_rmse")
RB_results_df

/Users/bunch/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


,Model,Best Parameters,Selected Features,Test_rmse
2,Neural Network,"{'model__alpha': 0.001, 'model__hidden_layer_sizes': (64, 32), 'model__learning_rate_init': 0.01}",22,60.658188
4,XGBoost,"{'model__colsample_bytree': 0.8, 'model__learning_rate': 0.01, 'model__max_depth': 3, 'model__n_estimators': 200, 'model__subsample': 0.5}",21,61.511702
1,Lasso,{'model__alpha': 10},21,62.320658
0,Random Forest,"{'model__max_depth': 5, 'model__min_samples_split': 10, 'model__n_estimators': 500}",18,63.011304
3,Support Vector Regression,"{'model__C': 0.1, 'model__gamma': 'scale', 'model__kernel': 'linear'}",19,67.972782


In [77]:
#RB predictions

best_model = grid.best_estimator_

best_features, test_rmse = backward_feature_elimination(best_model, X_train_RB, y_train_RB, X_test_RB, y_test_RB)

best_model.fit(X_train_RB[best_features], y_train_RB)

y_pred_RB = best_model.predict(X_test_RB[best_features])


RB_test_results = pd.DataFrame({
    "Actual_Points": y_test_RB,
    "Predicted_Points": y_pred_RB
}, index = X_test_RB.index)

RB_best_model = best_model
RB_best_features = best_features

RB_test_results = fantasy_data.loc[RB_test_results.index, ["Name", "Team", "Position"]].join(RB_test_results)
RB_test_results

,Name,Team,Position,Actual_Points,Predicted_Points
232,Brandon Bolden,NWE,RB,124.1,38.820644
608,Wayne Gallman,NYG,RB,13.5,83.586288
497,Trayveon Williams,CIN,RB,21.7,32.380714
1145,Rashaad Penny,SEA,RB,52.2,113.300812
241,Dare Ogunbowale,TAM,RB,29.9,61.072079
...,...,...,...,...,...
35,Josh Jacobs,OAK,RB,231.3,132.771240
648,Zack Moss,BUF,RB,105.2,91.234634
1110,AJ Dillon,GNB,RB,167.6,145.138885
360,Derek Watt,LAC,RB,0.0,26.684938


In [78]:
#WR Tuning

WR_results = []

for name, (base_model, param_grid) in model_pipeline.items():
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy = "median")),
        ("scaler", StandardScaler()),
        ("model", base_model)
        ])
    
    grid = GridSearchCV(pipe, param_grid = param_grid, scoring = rmse_scorer, cv = 3, n_jobs = -1)
    grid.fit(X_train_WR, y_train_WR)

    best_model = grid.best_estimator_
    best_parameters = grid.best_params_

    best_features, test_rmse = backward_feature_elimination(best_model, X_train_WR, y_train_WR, X_test_WR, y_test_WR)

    WR_results.append({
        "Model": name, 
        "Best Parameters": best_parameters,
        "Selected Features": len(best_features),
        "Test_rmse": test_rmse})
    
WR_results_df = pd.DataFrame(WR_results).sort_values("Test_rmse")
WR_results_df

/Users/bunch/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


,Model,Best Parameters,Selected Features,Test_rmse
0,Random Forest,"{'model__max_depth': 5, 'model__min_samples_split': 2, 'model__n_estimators': 200}",22,54.155282
4,XGBoost,"{'model__colsample_bytree': 0.5, 'model__learning_rate': 0.01, 'model__max_depth': 3, 'model__n_estimators': 350, 'model__subsample': 0.5}",22,54.276964
1,Lasso,{'model__alpha': 1},22,55.162495
3,Support Vector Regression,"{'model__C': 1, 'model__gamma': 'scale', 'model__kernel': 'linear'}",22,55.827516
2,Neural Network,"{'model__alpha': 0.01, 'model__hidden_layer_sizes': (128, 64), 'model__learning_rate_init': 0.01}",22,57.625581


In [79]:
#WR predictions

best_model = grid.best_estimator_

best_features, test_rmse = backward_feature_elimination(best_model, X_train_WR, y_train_WR, X_test_WR, y_test_WR)

best_model.fit(X_train_WR[best_features], y_train_WR)

y_pred_WR = best_model.predict(X_test_WR[best_features])


WR_test_results = pd.DataFrame({
    "Actual_Points": y_test_WR,
    "Predicted_Points": y_pred_WR
}, index = X_test_WR.index)

WR_best_model = best_model
WR_best_features = best_features

WR_test_results = fantasy_data.loc[WR_test_results.index, ["Name", "Team", "Position"]].join(WR_test_results)
WR_test_results

,Name,Team,Position,Actual_Points,Predicted_Points
421,Chris Moore,BAL,WR,0.0,17.911024
481,Jaydon Mickens,TAM,WR,12.8,19.956354
1545,Mason Kinsey,TEN,WR,1.3,21.787033
1748,Josh Reynolds,DET,WR,128.8,71.364517
1509,Kirk Merritt,MIA,WR,0.0,19.080769
...,...,...,...,...,...
974,Keith Kirkwood,CAR,WR,4.7,18.541893
1035,DeAndre Carter,2TM,WR,86.5,16.441494
685,Damiere Byrd,NWE,WR,64.9,96.674065
1510,Stanley Morgan Jr.,CIN,WR,0.0,19.291309


In [80]:
#TE Tuning

TE_results = []

for name, (base_model, param_grid) in model_pipeline.items():
    pipe = Pipeline([
        ("imputer", SimpleImputer(strategy = "median")),
        ("scaler", StandardScaler()),
        ("model", base_model)
        ])
    
    grid = GridSearchCV(pipe, param_grid = param_grid, scoring = rmse_scorer, cv = 3, n_jobs = -1)
    grid.fit(X_train_TE, y_train_TE)

    best_model = grid.best_estimator_
    best_parameters = grid.best_params_

    best_features, test_rmse = backward_feature_elimination(best_model, X_train_TE, y_train_TE, X_test_TE, y_test_TE)

    TE_results.append({
        "Model": name, 
        "Best Parameters": best_parameters,
        "Selected Features": len(best_features),
        "Test_rmse": test_rmse})
        
    
TE_results_df = pd.DataFrame(TE_results).sort_values("Test_rmse")
TE_results_df

/Users/bunch/opt/anaconda3/lib/python3.9/site-packages/sklearn/model_selection/_search.py:409: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  arr = np.array(param_list)


,Model,Best Parameters,Selected Features,Test_rmse
4,XGBoost,"{'model__colsample_bytree': 0.5, 'model__learning_rate': 0.01, 'model__max_depth': 3, 'model__n_estimators': 350, 'model__subsample': 0.8}",21,39.092130
0,Random Forest,"{'model__max_depth': 5, 'model__min_samples_split': 2, 'model__n_estimators': 350}",21,40.084302
1,Lasso,{'model__alpha': 1},21,41.346350
3,Support Vector Regression,"{'model__C': 1, 'model__gamma': 'scale', 'model__kernel': 'linear'}",21,42.733986
2,Neural Network,"{'model__alpha': 0.01, 'model__hidden_layer_sizes': (128,), 'model__learning_rate_init': 0.001}",22,43.320096


In [81]:
#TE predictions

best_model = grid.best_estimator_

best_features, test_rmse = backward_feature_elimination(best_model, X_train_TE, y_train_TE, X_test_TE, y_test_TE)

best_model.fit(X_train_TE[best_features], y_train_TE)

y_pred_TE = best_model.predict(X_test_TE[best_features])


TE_test_results = pd.DataFrame({
    "Actual_Points": y_test_TE,
    "Predicted_Points": y_pred_TE
}, index = X_test_TE.index)

TE_best_model = best_model
TE_best_features = best_features

TE_test_results = fantasy_data.loc[TE_test_results.index, ["Name", "Team", "Position"]].join(TE_test_results)
TE_test_results

,Name,Team,Position,Actual_Points,Predicted_Points
1390,Jesper Horsted,CHI,TE,4.9,15.104946
1956,Andrew Beck,DEN,TE,38.8,22.626835
2047,J.P. Holtz,NOR,TE,0.0,9.590635
1645,Dawson Knox,BUF,TE,54.6,103.407249
755,Cameron Brate,TAM,TE,78.5,45.615265
...,...,...,...,...,...
674,Dallas Goedert,PHI,TE,165.0,123.464188
1298,Josiah Deguara,GNB,TE,24.4,51.450897
220,Foster Moreau,OAK,TE,33.0,48.093697
1440,Luke Farrell,JAX,TE,8.0,22.591957


In [82]:
#validation

def predict_2024(model, features, X, y, X_2024):
    model.fit(X[features], y)

    return model.predict(X_2024[features])

QB_2024_pred = predict_2024(QB_best_model, QB_best_features, QB_data, QB_y, QB_data_2024)
RB_2024_pred = predict_2024(RB_best_model, RB_best_features, RB_data, RB_y, RB_data_2024)
WR_2024_pred = predict_2024(WR_best_model, WR_best_features, WR_data, WR_y, WR_data_2024)
TE_2024_pred = predict_2024(TE_best_model, TE_best_features, TE_data, TE_y, TE_data_2024)


In [114]:
def predict_2024(model, features, X, y, X_2024):
    model.fit(X[features], y)

    return model.predict(X_2024[features])


def metric_table(position, X_2024, y_pred):
    y_true = fantasy_data_2024.loc[X_2024.index, "Fantasy_PPR_NextYear"].copy()
    rmse = float(np.sqrt(mean_squared_error(y_true, y_pred)))
    mae = float(mean_absolute_error(y_true, y_pred))
    r2 = float(r2_score(y_true, y_pred))

    return {"Position": position, "RMSE": rmse, "MAE": mae, "R2": r2}


def position_prediciton_table(position, X_2024, predictions):
    table = fantasy_data_2024.loc[X_2024.index, ["Name", "Team", "Position", "Fantasy_PPR_NextYear"]].copy()
    table.rename(columns = {"Fantasy_PPR_NextYear": "Actual_PPR_2024"}, inplace = True)
    table["Predicted_PPR_2024"] = predictions
    table = table.sort_values("Predicted_PPR_2024", ascending = False).reset_index(drop = True)

    table["Predicted_Position_Rank"] = table["Predicted_PPR_2024"].rank(ascending = False, method = "first").astype(int)
    table["Actual_Position_Rank"] = table["Actual_PPR_2024"].rank(ascending = False, method = "first").astype(int)

    table = table.reindex(columns = ["Name", "Team", "Position", "Predicted_PPR_2024", "Actual_PPR_2024", "Predicted_Position_Rank", "Actual_Position_Rank"])

    return table


def overall_prediction_table(*position_tables):
    combined_table = pd.concat(position_tables, ignore_index = True)
    combined_table = combined_table.sort_values("Predicted_PPR_2024", ascending = False).reset_index(drop = True)

    combined_table["Predicted_Overall_Rank"] = combined_table["Predicted_PPR_2024"].rank(ascending = False, method = "first").astype(int)
    combined_table["Actual_Overall_Rank"] = combined_table["Actual_PPR_2024"].rank(ascending = False, method = "first").astype(int)

    combined_table = combined_table.reindex(columns = ["Name", "Team", "Position", "Predicted_PPR_2024", "Actual_PPR_2024", "Predicted_Overall_Rank", "Actual_Overall_Rank"])

    return combined_table


In [115]:
QB_2024_pred = predict_2024(QB_best_model, QB_best_features, QB_data, QB_y, QB_data_2024)
QB_metrics_table = pd.DataFrame([metric_table('QB', QB_data_2024, QB_2024_pred)])
QB_2024_table = position_prediciton_table("QB", QB_data_2024, QB_2024_pred)

print(QB_metrics_table)
QB_2024_table

  Position       RMSE        MAE        R2
0       QB  89.303701  69.120373  0.443413


,Name,Team,Position,Predicted_PPR_2024,Actual_PPR_2024,Predicted_Position_Rank,Actual_Position_Rank
0,Jalen Hurts,PHI,QB,284.797943,315.1,1,6
1,Lamar Jackson,BAL,QB,273.274963,430.4,2,1
2,C.J. Stroud,HOU,QB,255.809326,220.4,3,15
3,Josh Allen,BUF,QB,251.288651,379.0,4,2
4,Brock Purdy,SFO,QB,251.076065,266.9,5,11
...,...,...,...,...,...,...,...
59,Tim Boyle,NYJ,QB,45.025730,13.9,60,49
60,Cooper Rush,DAL,QB,44.319294,109.6,61,29
61,Mike White,MIA,QB,42.850891,1.1,62,56
62,Davis Mills,HOU,QB,42.749435,9.6,63,51


In [116]:
RB_2024_pred = predict_2024(RB_best_model, RB_best_features, RB_data, RB_y, RB_data_2024)
RB_metrics_table = pd.DataFrame([metric_table('RB', RB_data_2024, RB_2024_pred)])
RB_2024_table = position_prediciton_table("RB", RB_data_2024, RB_2024_pred)

print(RB_metrics_table)
RB_2024_table

  Position       RMSE       MAE        R2
0       RB  69.202078  51.66017  0.491197


,Name,Team,Position,Predicted_PPR_2024,Actual_PPR_2024,Predicted_Position_Rank,Actual_Position_Rank
0,Jahmyr Gibbs,DET,RB,200.311401,362.9,1,1
1,Kyren Williams,LAR,RB,196.813065,272.1,2,7
2,Christian McCaffrey,SFO,RB,189.663925,47.8,3,62
3,Travis Etienne,JAX,RB,186.939697,130.2,4,33
4,Saquon Barkley,NYG,RB,180.020157,355.3,5,2
...,...,...,...,...,...,...,...
100,Kenny McIntosh,SEA,RB,27.807093,22.4,101,81
101,DeeJay Dallas,SEA,RB,27.046961,21.0,102,82
102,Jeremy McNichols,SFO,RB,26.677626,63.8,103,54
103,Travis Homer,CHI,RB,24.724270,6.4,104,93


In [117]:
WR_2024_pred = predict_2024(WR_best_model, WR_best_features, WR_data, WR_y, WR_data_2024)
WR_metrics_table = pd.DataFrame([metric_table('WR', WR_data_2024, WR_2024_pred)])
WR_2024_table = position_prediciton_table("WR", WR_data_2024, WR_2024_pred)

print(WR_metrics_table)
WR_2024_table

  Position       RMSE        MAE        R2
0       WR  54.615466  42.032496  0.588802


,Name,Team,Position,Predicted_PPR_2024,Actual_PPR_2024,Predicted_Position_Rank,Actual_Position_Rank
0,Amon-Ra St. Brown,DET,WR,277.325073,316.2,1,3
1,Puka Nacua,LAR,WR,261.858337,206.6,2,23
2,A.J. Brown,PHI,WR,258.730469,216.9,3,17
3,Tyreek Hill,MIA,WR,254.382843,218.2,4,15
4,Ja'Marr Chase,CIN,WR,246.772369,403.0,5,1
...,...,...,...,...,...,...,...
179,Steven Sims,HOU,WR,17.657549,-2.0,180,183
180,Jaelon Darden,CLE,WR,17.029749,1.6,181,160
181,Cody Thompson,SEA,WR,16.660170,0.0,182,182
182,Collin Johnson,CHI,WR,16.536455,1.6,183,161


In [124]:
TE_2024_pred = predict_2024(TE_best_model, TE_best_features, TE_data, TE_y, TE_data_2024)
TE_metrics_table = pd.DataFrame([metric_table('TE', TE_data_2024, TE_2024_pred)])
TE_2024_table = position_prediciton_table("TE", TE_data_2024, TE_2024_pred)

print(TE_metrics_table)
TE_2024_table

  Position       RMSE        MAE        R2
0       TE  39.941916  28.343046  0.567785


,Name,Team,Position,Predicted_PPR_2024,Actual_PPR_2024,Predicted_Position_Rank,Actual_Position_Rank
0,Travis Kelce,KAN,TE,201.443619,195.4,1,4
1,Sam LaPorta,DET,TE,188.748535,174.6,2,7
2,Dalton Schultz,HOU,TE,167.825867,118.2,3,19
3,T.J. Hockenson,MIN,TE,165.853271,86.5,4,33
4,Evan Engram,JAX,TE,163.847687,89.5,5,32
...,...,...,...,...,...,...,...
106,Hunter Long,LAR,TE,10.399467,13.0,107,73
107,Anthony Firkser,DET,TE,10.275308,0.0,108,109
108,Rodney Williams,PIT,TE,10.082863,0.0,109,110
109,Brady Russell,SEA,TE,10.016452,0.0,110,111


In [126]:
All_metrics_table = pd.concat([QB_metrics_table, RB_metrics_table, WR_metrics_table, TE_metrics_table], ignore_index = True)
print(All_metrics_table)

All_results_2024 = overall_prediction_table(QB_2024_table, RB_2024_table, WR_2024_table, TE_2024_table)
All_results_2024

  Position       RMSE        MAE        R2
0       QB  89.303701  69.120373  0.443413
1       RB  69.202078  51.660170  0.491197
2       WR  54.615466  42.032496  0.588802
3       TE  39.941916  28.343046  0.567785


,Name,Team,Position,Predicted_PPR_2024,Actual_PPR_2024,Predicted_Overall_Rank,Actual_Overall_Rank
0,Jalen Hurts,PHI,QB,284.797943,315.1,1,13
1,Amon-Ra St. Brown,DET,WR,277.325073,316.2,2,12
2,Lamar Jackson,BAL,QB,273.274963,430.4,3,1
3,Puka Nacua,LAR,WR,261.858337,206.6,4,60
4,A.J. Brown,PHI,WR,258.730469,216.9,5,52
...,...,...,...,...,...,...,...
459,Hunter Long,LAR,TE,10.399467,13.0,460,342
460,Anthony Firkser,DET,TE,10.275308,0.0,461,455
461,Rodney Williams,PIT,TE,10.082863,0.0,462,456
462,Brady Russell,SEA,TE,10.016452,0.0,463,457
